In [52]:
from pytrends.request import TrendReq
from pytrends.exceptions import ResponseError
import os
import requests
import pandas as pd
from datetime import date, timedelta, datetime
import glob


pytrends = TrendReq(hl='en-US', tz = 240) #create pytrends object for api

In [53]:
start = date(2014,1,1) #must follow format YYYY, m, d
end = date(2019,5,16)

delta = end - start
times = []
for i in range(delta.days + 1): #returns list of every date between start and end 
    dt = start + timedelta(days=i)
    times.append(dt.strftime('%Y-%m-%d')) #convert datetime object into string and add to times ls

In [54]:
kw = ["Healthcare Jobs"] #this can be changed to anything, holds up to 5 terms seperated by commas 
ls = [] #empty list to begin building df
for i in range(0,len(times)-1): #iterates once for each day in date range between start and end 
    try:
        pytrends.build_payload(kw, timeframe=times[i] + ' ' + times[i+1], geo='US') #building google trends payload, setting region to US
        df = pytrends.interest_by_region(resolution = 'region') #get state level trends, return pandas dataframe
        date = str(times[i]) #wrap date in string 
        df.rename({str(kw[0]):date}, axis='columns', inplace= True) #name column with date of search
        ls.append(df.T) #add df to list of dfs. Transpose columns and rows (so that dates become rows and states become columns)
    #sleep(2)
    except (ResponseError, OSError) as e: #if google gets mad, don't lose everything we already gathered
        if len(ls) < 1: #if ls is empty there is no reason to export it to a csv
            print(e)
            
        else: #if list is not empty, save 
            print(e)
            ls = pd.concat(ls) #combine dataframes stored in ls
            fname = kw[0]+'.csv'
            folder = '\output\folder\if\something\goes\wrong'
            completename = os.path.join(folder, fname) #create complete file ouput name
            if os.path.exists(completename): #if filename already exists, create new name
                j = 0
                while os.path.exists(completename): #while a file exists with the desired name, add an increasing number to the filename 
                    j += 1
                    fname = kw[0] + "("+str(j)+").csv" #new filename with number at the end
                    completename = os.path.join(folder, fname)
                    
            ls.to_csv(completename)
            print('file saved')
        break



In [55]:
ls = pd.concat(ls) #Combine data from each date in specified range
ls.to_csv('path\to\desired\directory') #Save dataframe as csv in desired location

In [57]:
#If data for same search term is in several files, this will combine them 
def combine(path):
    os.chdir(path)
    extension = 'csv'
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))] #Collect list of all csv filenames
    
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ]) #Combine all csv in directory 
    combined_csv.to_csv( "combined_csv.csv", index=False) #Save as new csv name combined_csv.csv

In [59]:
combine('path\to\files\to\combine') #Run combine method if necessary 